<a href="https://colab.research.google.com/github/DeoZD/CSMODEL_G2_MCO/blob/main/CSMODEL_G2_MCO1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSMODEL MCO1 Group 2**
## LaSalleGameKNB?
* TIONGCO, KYAN THOMAS    S18
* DIAMANTE, DEO ZAMIR     S19
* LICUP, EVAN GABRIEL     S19
* SARROZA, MIKAEL JENSON	S19

### GamingStudy_data.csv
The original dataset consists of data collected as a part of a survey among gamers worldwide. The questionnaire asked questions that psychologists generally ask people who are prone to anxiety, social phobia, and less to no life satisfaction. The questionnaire consists of several set of questions as asked as a part of psychological study. The original data was collated by Marian Sauter and Dejan Draschkow.

In [ ]:
import pandas as pd
import numpy as np
import re

url = 'https://github.com/DeoZD/CSMODEL_G2_MCO/raw/refs/heads/main/GamingStudy_data.csv'
orig_df = pd.read_csv(url, encoding='latin-1')

## Original dataframe information:

In [ ]:
orig_df.info()

## Data Cleaning

In [ ]:
# @title Remove `highestleague` as it's empty
pre_df = orig_df.drop(columns=['highestleague'])

In [ ]:
# @title Remove `accept` as it's not indicative of what the survey data entails
pre_df = pre_df.drop(columns=['accept'])

**Note**: `accept` is meant to be the variable for Consent in survey participation, having either `Accept` or `NA` as values.

The value of `NA` means that the Consent step was skipped, as unlike the main study questions which were marked as required, the Consent part of the used survey form was not marked as such.

The used survey form states that not answering or not finishing the survey is the only way the data is not stored or shared.

Given that even if `accept` is `NA`, there are still participants *who answered the required parts of the survey until they finished*, we choose to interpret this as **"Consent by Action."**

By continuing to the next pages and answering the demographic and/or study-specific questions, the participant effectively demonstrated their willingness to participate.

The `NA` isn't treated as a "No"—just a skipped administrative step.

![accept](https://raw.githubusercontent.com/DeoZD/CSMODEL_G2_MCO/39481709c624a3431f3db90e0c1b8f9e033b5e3b/assets%20/accept.png)

In [ ]:
# @title Remove other irrelevant variables (`Reference`, `Timestamp`, `S. No.`)
pre_df = pre_df.drop(columns=['Reference', 'Timestamp', 'S. No.'])

In [ ]:
# @title Clean `League` categories

# Get the unique values to assess which values should stay or be removed
# pre_df['League'].unique()
# pre_df['League'].nunique()
# pre_df['League'].drop_duplicates().to_csv("unique_rank.csv", index=False)

In [ ]:
def extract_rank(x):
    rank_order = ['bronze', 'silver', 'gold', 'platinum', 'diamond', 'master', 'grandmaster', 'challenger']

    if pd.isna(x) or not str(x).strip():
        return np.nan

    x = str(x).lower().strip()

    # UNRANKED (PRIORITY)
    unranked_patterns = [
        r'\b(unranked|not\s+ranked|no\s+rank|unraked|unrankt|unrank)\b',
        r'\b(placement|provisional|seeding|qualifying|not\s+placed|still\s+placing)\b',
        r'\b(not\s+applicable|n/a|na|none)\b',
        r'\b(dont\s+play|don\'t\s+play|never\s+played|havent\s+played)\s+ranked\b',
        r'\b(havent|haven\'t)\s+(?:done|played)\s+(?:ranked|placement)\b',
        r'\b(under|pre|not)\s*(?:level|30|lvl)\b',
        r'\b(aram|normal|casual)\s+only\b',
        r'\b(too\s+toxic|rank\s+anxiety|anxiousness)\b',
        r'\b0\s*(?:games|ranked)\b',
    ]

    for pattern in unranked_patterns:
        if re.search(pattern, x):
            return 'unranked'

    found_ranks = set()

    # FULL WORD RANKS
    for rank in rank_order:
        if re.search(rf'\b{rank}\b', x):
            found_ranks.add(rank)

    # LETTER-NUMBER RANKS
    ln_map = {
        'b': 'bronze',
        's': 'silver',
        'g': 'gold',
        'p': 'platinum',
        'd': 'diamond',
        'm': 'master',
        'gm': 'grandmaster',
        'ch': 'challenger'
    }

    ln_matches = re.findall(r'\b(gm|ch|[bsgpdm])\s*\d+\b', x)
    for code in ln_matches:
        found_ranks.add(ln_map[code])

    # MISSPELLINGS / VARIANTS
    variations = {
        'bronze': ['bronz', 'brnz', 'broze', 'bronce'],
        'silver': ['silv', 'slvr', 'siver', 'sivler'],
        'gold': ['gld', 'glod', 'goled', 'golden'],
        'platinum': ['plat', 'pltn', 'platin', 'platen', 'platnium', 'platium'],
        'diamond': ['diam', 'diamon', 'diamomd'],
        'master': ['mstr', 'mst', 'masters'],
        'grandmaster': ['grandm', 'gmaster'],
        'challenger': ['chall', 'challngr', 'challen'],
    }

    for rank, vars_list in variations.items():
        for var in vars_list:
            if re.search(rf'\b{var}\b', x):
                found_ranks.add(rank)
                break

    # HISTORICAL CONTEXT
    if not found_ranks:
        for rank in rank_order:
            if re.search(rf'\b(was|last\s+season|previously)\s+{rank}\b', x):
                found_ranks.add(rank)

    # RETURN HIGHEST RANK
    if found_ranks:
        return max(found_ranks, key=lambda r: rank_order.index(r))

    return np.nan

### extract_rank(x)

function defined in order to clean up the very messy data of the 'League' variable column

## Logic Flow (in order)
I. **Input validation**
* If x is empty or `NaN` → return `NaN`
* Normalize text
* Lowercase + strip spaces
* Unranked detection (priority)

II. **Uses *regex patterns* to catch:**
1. **Unranked**
  - “unranked”, “not ranked”, “no rank”
  - placement/provisional
  - casual-only players
  - never played ranked
  - rank anxiety / toxicity
  - 0 ranked games

  → Immediately returns 'unranked'

**Special Case:** "NA/Not Applicable" or explicit declaration of League not being appilcable as stated in survey form is treated not as `NaN` or `NA` to differentiate from actual `NA` value being treated as 'Unknown' instead of 'Unranked'
- not applicable → returns 'unranked'

2. **Exact rank words**
* Matches whole words:
- bronze, silver, gold, platinum, diamond, master, grandmaster, challenger
3. **Letter-number ranks**
Converts codes like:
* B1 → bronze
* P5 → platinum
* GM1 → grandmaster
* CH1 → challenger

4. **Misspellings / variants**
* Catches typos like:
  * platnium → platinum
  * glod → gold
  * siver → silver
  * bronce → bronze

5. **Historical context**
* Detects phrases like:
  * "was diamond"
  * "last season gold"

III. **Resolution rule**
* If multiple ranks found → return the highest rank using predefined order

IV. **Fallback**
* If nothing matched → return NaN

In [ ]:
pre_df['League_clean'] = pre_df['League'].apply(extract_rank)

In [ ]:
# Filter rows where both 'League' and 'League_clean' are not NaN for display
filtered_df = pre_df[pre_df['League'].notna() & pre_df['League_clean'].notna()].copy()

# Get all unique clean league values to ensure all are covered
unique_clean_leagues = filtered_df['League_clean'].unique()

# Display examples for each unique clean league
for league in sorted(unique_clean_leagues):
    print(f"\nLeague_clean: {league.upper()}")
    display(filtered_df[filtered_df['League_clean'] == league][['League', 'League_clean']].head(3))
    print("--------------------------------------------------------------------------------------")


In [ ]:
# @title Clean `earnings` categories

# Get the unique values to assess which values should stay or be removed
# pre_df['earnings'].unique()
# pre_df['earnings'].nunique()
# pre_df['earnings'].drop_duplicates().to_csv("unique_earnings.csv", index=False)

In [ ]:
def extract_earnings(x):
    if pd.isna(x) or not str(x).strip():
        return np.nan

    x = str(x).lower().strip()

    # 1. MONETIZATION / INCOME
    monetization_keywords = [
        'earn', 'earning', 'money', 'paid', 'income', 'living', 'wage',
        'stream', 'streaming', 'youtube', 'tournament winnings',
        'betting', 'trading', 'tuition', 'side income', 'make money',
        'career', 'job', 'profitable', 'shoutcaster'
    ]
    if any(k in x for k in monetization_keywords):
        return 'Monetization'

    # 2. COMPETITIVE / PRO ASPIRATION
    competitive_keywords = [
        'competitive', 'competition', 'tournament', 'ranked',
        'climb', 'ladder', 'improve', 'improvement', 'better',
        'become', 'pro', 'professional', 'well known', 'best',
        'aspire', 'goal', 'achieve', 'rank 1'
    ]
    if any(k in x for k in competitive_keywords):
        return 'Competitive / Pro-Aspiration'

    # 3. BOOSTING (explicit)
    if 'boost' in x or 'eloboost' in x or 'boosting' in x:
        return 'Boosting'

    # 4. ESCAPISM
    escapism_keywords = [
        'escape', 'forget', 'real life', 'get away', 'relief',
        'fill the void', 'numb', 'suppress', 'mental', 'memories'
    ]
    if any(k in x for k in escapism_keywords):
        return 'Escapism'

    # 5. ADDICTION (explicit psychological dependence)
    addiction_keywords = [
        'addicted', 'addiction', 'can’t stop', "can't stop",
        'compulsion', 'dependent', 'hooked'
    ]
    if any(k in x for k in addiction_keywords):
        return 'Addiction'

    # 6. HABIT (automatic behavior, routine)
    habit_keywords = [
        'habit', 'routine', 'autopilot', 'used to', 'just play',
        'normally play', 'always play', 'keep playing'
    ]
    if any(k in x for k in habit_keywords):
        return 'Habit'

    # 7. BOREDOM (time-filling behavior)
    boredom_keywords = [
        'bored', 'nothing better to do', 'kill time',
        'pass time', 'spend my time', 'time somehow',
        'waste time', 'no work', 'no job'
    ]
    if any(k in x for k in boredom_keywords):
        return 'Boredom'

    # 8. FUN & SOCIAL (default hobby class)
    fun_keywords = [
        'fun', 'friends', 'social', 'hobby',
        'enjoy', 'love', 'passion', 'relax'
    ]
    if any(k in x for k in fun_keywords):
        return 'Fun & Social'

    # 9. FALLBACK
    return 'Other / Just Playing'

### extract_earnings(x)

function defined in order to clean up and recategorize the data of the `earnings` variable

In [ ]:
pre_df.info()

In [ ]:
spin_vars = [f'SPIN{i}' for i in range(1, 18)]

# Show counts of nulls for just these columns
null_counts = pre_df[spin_vars].isnull().sum()
print(null_counts)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.heatmap(pre_df[spin_vars].isnull(), yticklabels=False, cbar=False, cmap='viridis')
plt.title('Missing Data Pattern for SPIN Variables')
plt.show()